In [1]:
import pandas as pd
import re
from os import walk
import random
import shutil
import PyPDF2
import pikepdf
from contextlib import contextmanager
from contextlib import contextmanager
import threading
import _thread

In [2]:
df00 = pd.read_csv("~/Downloads/form_data.csv")

In [3]:
df00

,title,group,list_codes,pages,f_per_p,reading_level,downloaded_on,original_url,original_fields,new_url,standardized_fields,jurisdiction,text
0,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,[],2.0,7.500000,10.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,"['name__1', 'name__2', 'docket_number', 'users...",https://courtformsonline.org/forms/04b3a073477...,"['name__1', 'name__2', '*docket_number', '*use...",UT,COMMUNITY SERVICE WORKSHEET\n THIRD DISTRICT J...
1,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,['GO-00-00-00-00'],2.0,19.000000,10.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,"['docket_number', 'users1_birthdate', 'male', ...",https://courtformsonline.org/forms/6e420f1b357...,"['*docket_number', '*users1_birthdate', 'male'...",UT,THIRD DISTRICT JUVENILE COURT WORK PROGRAM REF...
2,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,[],5.0,4.800000,12.5,2021-11-11,https://www.utcourts.gov/courts/juv/juvsites/3...,['of the probation department or by order of t...,https://courtformsonline.org/forms/2532cd2b6d3...,"['probation_department_order_court', 'special_...",UT,Page 1 of 4\n UTAH STATE DISTRICT JUVENILE COU...
3,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,[],3.0,14.666667,6.5,2021-11-11,https://www.utcourts.gov/howto/family/adoption...,"['Name', 'Address', 'City State Zip', 'Phone',...",https://courtformsonline.org/forms/f6a6814890f...,"['*users1_name', '*users1_address_line_one', '...",UT,Consent to Adoption and Waiver of \n Rights\n ...
4,Adoption Agreement,Adopting a Minor Stepchild,['GO-00-00-00-00'],1.0,19.000000,7.5,2021-11-11,https://www.utcourts.gov/howto/family/adoption...,"['Name', 'Address', 'City State Zip', 'Phone',...",https://courtformsonline.org/forms/be656a9a361...,"['*users1_name', '*users1_address_line_one', '...",UT,Adoption\n \n Agreement\n \n Approved Board of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,Order Granting Motion to Terminate Stalking Or...,Protection Orders,"['CO-07-00-00-00', 'CO-00-00-00-00']",1.0,11.000000,12.5,2022-03-28,https://www.courts.state.wy.us/wp-content/uplo...,"['Name', 'ST', 'Name_2', 'Terminate Order of P...",https://courtformsonline.org/forms/3649f8e39b4...,"['*users1_name', 'st', 'name', 'terminate_orde...",WY,Order Granting Motion to Terminate Page 1 of 1...
10014,Order Denying Motion to Terminate Stalking Ord...,Protection Orders,"['CO-07-00-00-00', 'CO-00-00-00-00']",1.0,9.000000,7.5,2022-03-28,https://www.courts.state.wy.us/wp-content/uplo...,"['Name', 'ST', 'Name_2', 'undefined', 'remains...",https://courtformsonline.org/forms/82de0286f46...,"['*users1_name', 'st', 'name', 'undefined', 'r...",WY,Order Denying Motion to Terminate Page 1 of 1 ...
10015,Stalking Motion and Affidavit to Show Cause,Protection Orders,[],2.0,7.500000,9.5,2022-03-28,https://www.courts.state.wy.us/wp-content/uplo...,"['Name', 'ST', 'Name_2', '2 Respondent violate...",https://courtformsonline.org/forms/6392da792d3...,"['*users1_name', 'st', 'name', 'respondent_vio...",WY,Motion and Affidavit for Order . to Show Cause...
10016,Order to Show Cause in Stalking Cause,Protection Orders,"['FA-07-00-00-00', 'CO-07-00-00-00', 'CR-00-00...",2.0,4.000000,10.5,2022-03-28,https://www.courts.state.wy.us/wp-content/uplo...,"['Name', 'ST', 'Name_2', 'Based on the records...",https://courtformsonline.org/forms/b11a0c83c9a...,"['*users1_name', 'st', 'name', 'based_records_...",WY,Stalking Order to Show Cause Page 1 of 1 . Rev...


In [2]:
import formfyxer as formfyxer

In [3]:
class TimeoutException(Exception): pass
@contextmanager
def time_limit(seconds):
    timer = threading.Timer(seconds, lambda: _thread.interrupt_main())
    timer.start()
    try:
        yield
    except KeyboardInterrupt:
        raise TimeoutException("Timed out.")
    finally:
        # if the action ends in specified time, timer is canceled
        timer.cancel()

        
def get_fields(fileloc):
    f = PyPDF2.PdfFileReader(fileloc)

    if f.isEncrypted:
        pdf = pikepdf.open(fileloc, allow_overwriting_input=True)
        pdf.save(fileloc)
        f = PyPDF2.PdfFileReader(fileloc)

    npages = f.getNumPages()

    # When reading some pdfs, this can hang due to their crazy field structure
    try:
        with time_limit(15):
            ff = f.getFields()
    except TimeoutException as e:
        print("Timed out!")
        ff = None   

    if ff:
        fields = list(ff.keys())
    else:
        fields = []
        
    return fields

In [95]:
max = 200

path = "../data/raw/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(dirnames)
    break
f1 = [file for file in f]

n = round(max/len(f1))
print(n)

i = 0

for folder in f1:
    path_too = path+folder+"/forms/"
    print(path_too)
    f_ = []
    for (dirpath, dirnames, filenames) in walk(path_too):
        f_.extend(filenames)
        break
    f2 = [file for file in f_ if re.search(".*\.pdf",file)]
    
    rand = random.choices(f2, k=n)
    for file in rand:
        shutil.copy(path_too+file, "../data/random/"+str(i)+"_from_run_one.pdf")
        i+=1

8
../data/raw/courts.state.wy.us/forms/
../data/raw/www.courts.michigan.gov/forms/
../data/raw/courts.delaware.gov/forms/
../data/raw/www.nccourts.gov/forms/
../data/raw/www.vermontjudiciary.org/forms/
../data/raw/www.wicourts.gov/forms/
../data/raw/www.courts.state.md.us/forms/
../data/raw/courts.alaska.gov/forms/
../data/raw/eforms.alacourt.gov/forms/
../data/raw/www.mass.gov/forms/
../data/raw/ujslawhelp.sd.gov/forms/
../data/raw/courtselfhelp.idaho.gov/forms/
../data/raw/mjbportal.courts.maine.gov/forms/
../data/raw/www.vacourts.gov/forms/
../data/raw/selfhelp.nvcourts.gov/forms/
../data/raw/www.utcourts.gov/forms/
../data/raw/www.courts.state.co.us/forms/
../data/raw/www.jud.ct.gov/forms/
../data/raw/supremecourt.nebraska.gov/forms/
../data/raw/www.courts.state.hi.us/forms/
../data/raw/www.courts.ca.gov/forms/
../data/raw/kansasjudicialcouncil.org/forms/
../data/raw/www.mncourts.gov/forms/
../data/raw/www.courts.wa.gov/forms/
../data/raw/www.illinoiscourts.gov/forms/


In [4]:
path = "../data/random/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

for file in f1:
    if re.search(".*\.pdf",file):
        #print(file)
        try:
            fields = get_fields( '../data/random/'+file)
            #print(fields)
            if fields==[]:
                #try:
                formfyxer.auto_add_fields(path+file, '../data/random_proc/'+file)
                print(file)
                print("Processed")
                #except:
                #    shutil.copy(path+file, "../data/random_proc/"+file)
                #    print("Error 1!!!!!!")
            else:
                shutil.copy(path+file, "../data/random_proc/"+file)
                print("No changes made.")
        except: 
            shutil.copy(path+file, "../data/random_proc/"+file)
            print("Error 2!!!!!!")


No changes made.
Error 2!!!!!!
No changes made.
No changes made.
No changes made.
No changes made.
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp9qtp0n8q
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpsudqeucn
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpxlw8x8m4
[]
Choices: [(4.585469938839457, 'Name \n'), (29.50988021934349, 'Address \n')]
Choices: [(29.50988021934349, 'City, State, Zip \n'), (18.37253272034095, 'Name \n'), (4.58546993883942, 'Address \n')]
Choices: [(4.585469938839457, 'City, State, Zip \n'), (45.77817233747967, 'Name \n'), (18.37253272034095, 'Address \n'), (29.509880219343604, 'Phone \n'), (216.93192057417465, 'Check your email. You will receive information and \n'), (149.34366944735223, 'documents at this email address.  \n')]
Choices: [(18.37253272034095, 'City, State, Zip \n'), (45.77817233747978, 'Address \n'), (4.585469938839457, 'Phone \n'), (32.12347089839463, 'Email  \n'), (213.7

Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpyw4g_sed
[]
Choices: [(118.38824007983227, 'Attachment to Petition for Order for Protection\n'), (13.017479808319331, 'Continued from Question 4 of the Petition \n'), (12.2675282176157, '4. Other Minor Child \n')]
Choices: [(49.002142611930815, 'Continued from Question 4 of the Petition \n'), (11.722800713566775, '4. Other Minor Child \n'), (29.855595520103087, 'Name: (first, middle, last)\n')]
Choices: [(131.77692838983612, 'Name: (first, middle, last)\n'), (132.08160844406007, 'Date of birth: (month/day/year)\n'), (135.0610564989405, 'This child is currently living with \n'), (61.04062315548556, ' Me   \n'), (114.07090589506164, 'Respondent\n')]
Choices: [(163.4389995315071, 'Name: (first, middle, last)\n'), (162.01692314437398, 'Date of birth: (month/day/year)\n'), (162.2144379967763, 'This child is currently living with \n'), (24.486458200605483, ' Me   \n'), (80.3223480964047, 'Respondent\n')]
Choices: [(86.4193

[[(301, 941, 36, 36) False
  array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0, 255,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   

[[(200, 466, 37, 37) False
  array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0

[[(361, 270, 34, 32) False
  array([[255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0, 255,   0,   0,   0],
         [255,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0, 255,   0,   0, 255],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0, 255,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
            0,   0,   0,   0,   0,   0,   0,   0,   

...

No changes made.
No changes made.
No changes made.
Error 2!!!!!!
No changes made.
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp4b7_5g8d
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp9juh0e1t
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp94sri6oa
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp74qnixp7
[]
Choices: [(26.217126921155916, 'IN THE COURT OF CHANCERY FOR THE STATE OF DELAWARE \n'), (2.778413936043388, 'C.M.# ___________________ \n'), (78.59753421068629, 'ANNUAL UPDATE & MEDICAL STATEMENT \n')]
Choices: [(59.62254727869313, 'IN THE COURT OF CHANCERY FOR THE STATE OF DELAWARE \n'), (2.0627942214385238, '________________________________________, \n'), (197.62794433986303, 'C.M.# ___________________ \n'), (3.7248248280959486, 'A person with a disability  \n'), (17.40406619155416, 'Date of birth: _______________________ \n'), (120.7844807249673, 'ANNUAL UPDATE & MEDICAL STATEMEN

22_from_run_one.pdf
Processed
No changes made.
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpcxweuv4p
[]
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmp1hqc8yex
[]
Choices: [(121.76344564693626, 'STATE OF WISCONSIN, CIRCUIT COURT,            \n \n'), (175.15587072951453, 'IN THE MATTER OF THE ESTATE OF \n \n'), (47.3444049070215, ' COUNTY \n'), (45.788249540684546, 'Waiver and Consent \n'), (65.27361086993724, '(Informal Administration) \n'), (13.364746109073602, ' Amended \n')]
Choices: [(3.114826890856066, 'Name \n \n'), (27.56956188552877, 'STATE OF WISCONSIN, CIRCUIT COURT,            \n \n'), (9.60928418561967, 'IN THE MATTER OF THE ESTATE OF \n \n'), (185.25969350077207, '(Informal Administration) \n'), (124.53868105355863, 'Case No.            \n')]
Choices: [(10.215649297034457, 'Name \n \n'), (26.493621997756332, '  1. \n \n'), (46.52413711784458, '  2. \n \n'), (36.56435344102233, 'IN THE MATTER OF THE ESTATE OF \n \n'), (184.407

47_from_run_one.pdf
Processed
No changes made.
No changes made.
Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpqxsv5wq1
[]
Choices: [(62.75989272266166, 'County Court _________________________ County, Colorado \n'), (62.82334272036152, 'Court Address: \n'), (71.25782712488504, 'State of Colorado \n')]
Choices: [(23.46988484334763, 'County Court _________________________ County, Colorado \n'), (11.931321073544215, 'Court Address: \n'), (7.876025810013633, 'State of Colorado \n'), (24.749808050972902, 'v. \n'), (40.0324705028306, 'Defendant: _______________________________________  \n    \n'), (147.81778849047362, 'COURT USE ONLY \n')]
Choices: [(60.805634003437554, 'State of Colorado \n'), (54.876281566447254, 'v. \n'), (33.592931277493534, 'Defendant: _______________________________________  \n    \n'), (21.142013333985066, 'Attorney or Party Without Attorney (Name and Address):  \n')]
Choices: [(144.2663874454476, 'ETHYL ALCOHOL OR MARIJUANA BY AN UNDERAGE PERSO

Processing file:  /var/folders/d7/2h1shmhj4ks536wzvbblxfnw0000gn/T/tmpjyyscjps
[]
Choices: [(27.02178363913097, '\xa0\n'), (44.14075667679483, '\xa0\xa0\xa0\xa0\xa0\xa0IN\xa0CIRCUIT\xa0COURT\xa0\n'), (1.89736659610088, '_________\xa0JUDICIAL\xa0CIRCUIT\xa0\n'), (92.09227667942625, '******************************************************************************\xa0\n'), (59.90077515358197, 'CIV.\xa0_________________\xa0\n'), (45.87148775612151, '\xa0\n'), (37.11639784461847, '\xa0\n')]
Choices: [(42.62872440033842, 'STATE\xa0OF\xa0SOUTH\xa0DAKOTA\xa0\n'), (2.2146746939447772, 'COUNTY\xa0OF\xa0_________________\xa0\n'), (64.05193205516909, '******************************************************************************\xa0\n'), (16.412122524524435, '\xa0\n'), (49.375100665821265, '\xa0\n'), (48.40116375625695, '\xa0\n'), (67.16558235405972, '\xa0\n'), (43.10249162171469, 'In\xa0the\xa0Matter\xa0of\xa0the\xa0Petition\xa0regarding\xa0\n'), (35.65834073537363, ')\xa0\n'), (23.344182658641095,

No changes made.
No changes made.


In [73]:
get_fields("../data/random/0cad14d53074dca6ba097de6a8ca5a49.pdf")

[]